**Modelo tipo 1 e 2**

* Tipo 1: demanda <= oferta

* Tipo 2: demanda >= oferta

* fim no cliente

In [37]:
import gurobipy as gp
import numpy as np
import pandas as pd
from math import *

In [38]:
df_ori_dest = pd.read_csv('../dados/origem_porto.csv')
df_ori_trans = pd.read_csv('../dados/origem_transbordo.csv')
df_trans_porto = pd.read_csv('../dados/transbordo_porto.csv')

In [39]:
df_supply = pd.read_csv('../dados/supply.csv')
df_cap_port = pd.read_csv('../dados/cap_porto.csv')
df_cap_trans = pd.read_csv('../dados/cap_transbordo.csv')
df_demand = pd.read_csv('../dados/demand.csv')

In [40]:
qnt_orig = df_supply.shape[0]
qnt_trans = df_cap_trans.shape[0]
qnt_port = df_cap_port.shape[0]
qnt_cli = df_demand.shape[0]

In [41]:
df_supply

,0
0,851
1,630
2,532
3,467
4,278
5,159
6,116
7,748
8,617
9,473


In [42]:
df_demand

,0
0,3238
1,1911
2,722
3,2540
4,1989
5,4212
6,456
7,2041
8,1261
9,3545


In [43]:
df_ori_dest

,0,1,2,3,4,5,6,7,8,9
0,106.436445,104.168848,80.220496,128.348526,116.580616,66.375899,131.341387,153.999896,126.723636,44.018905
1,105.875519,105.412804,81.187979,127.844906,117.817377,66.419852,130.832852,154.802046,126.972677,42.725093
2,105.464762,105.050693,80.678245,127.415566,117.447956,65.868912,130.405104,154.253443,126.402127,42.810989
3,97.936992,109.949807,82.572712,120.078907,122.208373,61.305269,123.048218,154.474593,122.803518,35.417510
4,103.826317,107.019430,81.930312,125.862554,119.391524,65.269411,128.844017,155.110985,126.151655,40.387246
5,111.470405,100.873116,79.169610,133.235431,113.349696,69.691893,136.239319,153.814018,129.159251,49.384046
6,102.642338,105.643870,80.141475,124.609187,117.991186,63.503071,127.597066,153.219863,124.294875,41.001541
7,102.667775,107.260878,81.718659,124.710635,119.612066,64.311442,127.691328,154.688477,125.291443,39.644177
8,106.028343,104.616266,80.485725,127.960494,117.022145,66.196894,130.951667,154.175256,126.634011,43.448471
9,102.566115,106.931365,81.360905,124.593470,119.279973,64.056375,127.575697,154.336650,125.005165,39.876931


In [44]:
N = [i for i in range(qnt_orig)]
M = [i + qnt_orig + qnt_trans for i in range(qnt_port)]
K = [i + qnt_orig for i in range(qnt_trans)]
O = [i + qnt_orig + qnt_trans + qnt_port for i in range(qnt_cli)]

supply = {}
for i in N:
    supply[i] = df_supply['0'][i]

demand = {}
for i in O:
    demand[i] = df_demand['0'][i - O[0]]

cap_transbordo = {}
for i in K:
    cap_transbordo[i] = df_cap_trans['0'][i - K[0]]

cap_port = {}
for i in M:
    cap_port[i] = df_cap_port['0'][i - M[0]]

cost = {}
for i in N:
    for j in M:
        cost[i, j] = df_ori_dest[str(j - M[0])][i]
    for k in K:
        cost[i, k] = df_ori_trans[str(k - K[0])][i]

for j in M:
    for k in K:
        cost[k, j] = df_trans_porto[str(j - M[0])][k - K[0]]
    for o in O:
        cost[j, o] = 0


In [45]:
oferta_total = 0
for i in supply:
    oferta_total += supply[i]
demanda_total = 0
for i in demand:
    demanda_total += demand[i]

print(oferta_total, demanda_total)

4871 21915


In [46]:
m = gp.Model("probrema")

In [47]:
X = {}
for i in N:
    for j in M:
        X[i, j] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="supply_{}_port_{}".format(i, j))

for i in N:
    for k in K:
        X[i, k] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="supply_{}_tranship_{}".format(i, k))
                
for j in M:
    for k in K:
        X[k, j] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="tranship_{}_port_{}".format(k, j))

for j in M:
    for o in O:
        X[j, o] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="X_{}_{}".format(j, o))

In [48]:
m.setObjective(
    gp.quicksum((X[i, j] * cost[i, j]) for i in N for j in M) + \
    gp.quicksum((X[i, k] * cost[i, k]) for i in N for k in K) + \
    gp.quicksum((X[k, j] * cost[k, j]) for j in M for k in K),
    sense=gp.GRB.MINIMIZE
)

In [49]:
if oferta_total <= demanda_total:
    for i in N:
        m.addConstr(
            (gp.quicksum(X[i, j] for j in M) + gp.quicksum(X[i, k] for k in K)) == supply[i]
        )
else:
    for i in N:
        m.addConstr(
            (gp.quicksum(X[i, j] for j in M) + gp.quicksum(X[i, k] for k in K)) <= supply[i]
        )

In [50]:
if oferta_total <= demanda_total:
    for o in O:
        m.addConstr(
            gp.quicksum(X[j, o] for j in M) <= demand[o]
        )
else:
    for o in O:
        m.addConstr(
            gp.quicksum(X[j, o] for j in M) == demand[o]
        )

In [51]:
for j in M:
    m.addConstr(
        (gp.quicksum(X[i, j] for i in N) + gp.quicksum(X[k, j] for k in K)) <= cap_port[j]
    )

for k in K:
    m.addConstr(
        gp.quicksum(X[i, k] for i in N) <= cap_transbordo[k]
    )

for k in K:
    m.addConstr(
        gp.quicksum(X[i, k] for i in N) == gp.quicksum(X[k, j] for j in M)
    )

for j in M:
    m.addConstr(
        gp.quicksum(X[j, o] for o in O) == (gp.quicksum(X[i, j] for i in N) + gp.quicksum(X[k, j] for k in K))
    )

In [52]:
rest = m.addConstrs(
    gp.quicksum(X[i, j] for i in N) >= 0 for j in M
)

rest = m.addConstrs(
    gp.quicksum(X[i, k] for i in N) >= 0 for k in K
)

rest = m.addConstrs(
    gp.quicksum(X[k, j] for k in K) >= 0 for j in M
)

In [53]:
# Executa o modelo
m.update()
m.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 90 rows, 400 columns and 1400 nonzeros
Model fingerprint: 0x9c5fc652
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 1e+04]
Presolve removed 30 rows and 0 columns
Presolve time: 0.03s
Presolved: 60 rows, 400 columns, 1100 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.5946274e+04   1.058195e+03   0.000000e+00      0s
      57    2.2740267e+05   0.000000e+00   0.000000e+00      0s

Solved in 57 iterations and 0.04 seconds (0.00 work units)
Optimal objective  2.274026695e+05


In [54]:
m.write("out.sol")